<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Histogram function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'> <p style = 'font-size:16px;font-family:Arial'>
    The <b>Histogram()</b> function calculates the frequency distribution of a data set
    using any of these methods:
<ul style = 'font-size:16px;font-family:Arial'><li>Sturges</li>
    <li>Scott</li>
    <li>Variable-width</li>
    <li>Equal-width</li></ul> <p style = 'font-size:16px;font-family:Arial'>In this notebook we will see how we can use the Histogram function available in Vantage.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_TitanicSurvival_cloud');"        # Takes 30 seconds
#%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_TitanicSurvival_local');" 

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
tdf = DataFrame(in_schema("DEMO_TitanicSurvival", "Passenger_Data"))
print("Shape of the data: ", tdf.shape)
tdf

In [ ]:
tdf.tdtypes

In [ ]:
help(Histogram)

In [ ]:
#Get the frequency distribution of a data set using 'STURGES' method type for the values in column 'age'.
hist_obj1 = Histogram(data=tdf,
                     target_columns="age",
                     method_type="STURGES")
hist_obj1.result

In [ ]:
plot_age=hist_obj1.result
plot_age.plot(x=plot_age.Label,y=plot_age.Bin_Percent,kind='bar')

In [ ]:
# Get the frequency distribution of a data set with respect
# to 'sex' column using 'EQUAL-WIDTH' method type for the
# values in column 'age' with 10 and 2 number
#            of bins respectively.
hist_obj2 = Histogram(data=tdf,
                     target_columns=["age"],
                     method_type="EQUAL-WIDTH",
                     nbins=[10],
                     groupby_columns=["sex"])
 
# Print the result DataFrame.
plot_age2= hist_obj2.result
plot_age2.sort('Label')

In [ ]:
plot_age2 = plot_age2.assign(Count_female = case([(plot_age2.sex == 'female', plot_age2.CountOfValues)], else_ = 0),
                                Count_male = case([(plot_age2.sex == 'male', plot_age2.CountOfValues)], else_ = 0))
plot_age2

In [ ]:
plot_age_sex= plot_age2.select(['Label','Count_female','Count_male']).groupby('Label').sum()
plot_age_sex

In [ ]:
plot_age_sex.plot(x=plot_age_sex.Label,y=[plot_age_sex.sum_Count_female,plot_age_sex.sum_Count_male],kind='bar')

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. Cleanup</b>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_TitanicSurvival');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>Histogram function reference: <a href = 'https://docs.teradata.com/search/all?query=TD_Histogram&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>